In [ ]:
using Pkg
Pkg.activate("D:\\Science\\Code\\Julia\\envr")

In [1]:
using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie

In [2]:
function TM(u, p, t)

    U(y) = U0 + ΔU0 / ( 1 + exp( -50*(y-ythr) ) )
    σ(x) = 1 / ( 1 + exp( -20*(x-xthr) ) )

    E, x, y = u
    α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0 = p

    du1 = (-E + α * log( 1 + exp( (J * U(y) * x * E + I0  ) / (α) ) ) ) / τ
    du2 = (1-x)/τD - U(y)*x*E
    du3 = (-y)/τy+β*σ(x)
    return SVector(du1, du2, du3)
end;

In [23]:
function output(idx_I0, idx_U0, I0, U0, u0)
    println("Индекс I0: $idx_I0; I0: $I0");flush(stdout)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_U0,U0, u0)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output_end_iter(ΛΛ, u0_lc)
    println("Спектр: $(ΛΛ)");flush(stdout)
    println("Последняя точка: $(u0_lc)");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [52]:
t = 150.0; tt = 250.0; tstep = 0.001
trange = range(0.0, t, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [53]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.3;  const ΔU0 = 0.305; I0 = -1.40;

In [54]:
u0 = SA[17.6055, 0.6181, 0.429232]

3-element SVector{3, Float64} with indices SOneTo(3):
 17.6055
  0.6181
  0.429232

In [55]:
I0range = range( -1.40, -1.87036, length = 50 )
U0range = range( 0.3,  0.5, length = 50)

0.3:0.004081632653061225:0.5

In [56]:
Λs = zeros(length(I0range), length(U0range), 3)
u0s = zeros(length(I0range), length(U0range), 3);

In [57]:
function init_ds(U0_, u0_lc)
    p = [α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0range[1]]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function goto_attractor(ds_)
    tr = trajectory(ds_, t; Ttr = tt, diffeq = integ_set)
    u0_lc = tr[end]
    return u0_lc
end
function spectrum(ds_)
    ΛΛ = lyapunovspectrum(ds_, t; diffeq = integ_set)
    return ΛΛ
end
function save_out(idx_U0, ΛΛ, u0_lc)
    Λs[1, idx_U0, :] = ΛΛ
    u0s[1, idx_U0, :] = u0_lc
end

save_out (generic function with 1 method)

In [58]:
for (idx_U0, U0_) in enumerate(U0range)
    
    if idx_U0 == 1 # Проверка на первую итерацию
        global u0_lc = u0
    end
    
    output(idx_U0, U0_, u0_lc) # Вывод
    
    ds = init_ds(U0_, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds) # Выход на аттрактор
    ΛΛ = spectrum(ds) # Вычисление спектра
    save_output(idx_U0, ΛΛ, u0_lc) # Сохранение значений
    output_end_iter(ΛΛ, u0_lc) # Вывод
    separate()
    
end

Индекс U0: 1; U0: 0.3
Начальные условия: [17.6055, 0.6181, 0.429232]
Спектр: [0.01807156198337373, -2.7570033617871093, -2.8143517162816933]
Последняя точка: [6.128004599054316, 0.7187832580003417, 0.41649927269079234]

---------
Индекс U0: 2; U0: 0.3040816326530612
Начальные условия: [6.128004599054316, 0.7187832580003417, 0.41649927269079234]
Спектр: [0.032243811272423976, -2.7876175163094477, -2.827712963022294]
Последняя точка: [13.431412171635934, 0.6046280515264753, 0.42313574330643194]

---------
Индекс U0: 3; U0: 0.3081632653061224
Начальные условия: [13.431412171635934, 0.6046280515264753, 0.42313574330643194]
Спектр: [0.03309306268498603, -2.8076123014198306, -2.864184369771768]
Последняя точка: [9.305250739332083, 0.6451506361176265, 0.4175739019180914]

---------
Индекс U0: 4; U0: 0.3122448979591837
Начальные условия: [9.305250739332083, 0.6451506361176265, 0.4175739019180914]
Спектр: [0.029236414889324994, -2.8584662282539877, -2.8653457663261603]
Последняя точка: [4.10507

Последняя точка: [13.255325205491628, 0.5874927014154064, 0.3919801591817549]

---------
Индекс U0: 31; U0: 0.42244897959183675
Начальные условия: [13.255325205491628, 0.5874927014154064, 0.3919801591817549]
Спектр: [0.03020185390765633, -3.088892474163964, -3.126450787120752]
Последняя точка: [19.277732420305085, 0.589608729712729, 0.3956733617872022]

---------
Индекс U0: 32; U0: 0.42653061224489797
Начальные условия: [19.277732420305085, 0.589608729712729, 0.3956733617872022]
Спектр: [0.026673253597534242, -3.0593664406714725, -3.1386996620263408]
Последняя точка: [12.545972569627228, 0.5970115834899047, 0.3887310390512674]

---------
Индекс U0: 33; U0: 0.4306122448979592
Начальные условия: [12.545972569627228, 0.5970115834899047, 0.3887310390512674]
Спектр: [0.03363014690626831, -3.06990976331789, -3.115693694570517]
Последняя точка: [6.998193154428695, 0.686166781108289, 0.38191103956692435]

---------
Индекс U0: 34; U0: 0.4346938775510204
Начальные условия: [6.998193154428695, 0.

In [ ]:
lyapmin,lyapmax = minimum(Λs[:, :, 3]), maximum( Λs[:, :, 3])

f = Figure()
ax1 = Axis(f[1, 1], xlabel = L"k_1",ylabel = L"k_2", xlabelsize = 50, ylabelsize = 50,
            xticklabelsize = 35, yticklabelsize = 35,
            xgridvisible  = false, ygridvisible = false)
hm = heatmap!(ax1, I0range, U0range, Λs[:, :, 1], colormap = :seismic)
Colorbar(f[1, 2], hm)
f